In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # Not used in this consolidated script's printouts
import seaborn as sns # Not used in this consolidated script's printouts

# --- Cell 1: Setup (Assuming BigQuery client 'client' is already set up) ---
# This part should be run once to load your initial data.
# If running this as a whole new script, you need to execute your BigQuery loading logic first.
# Ensure df_companyaccounts, df_leads, etc., are loaded before proceeding.
# Example (from your notebook, run this if DataFrames are not loaded):
from google.colab import auth
auth.authenticate_user()
!pip install --quiet google-cloud-bigquery pandas openpyxl
from google.cloud import bigquery
PROJECT_ID = "lead-conversion-prediction"
DATASET = "lead_conversion_staging_tables"
client = bigquery.Client(project=PROJECT_ID)
def load_table(table_name):
    query = f"SELECT * FROM `{PROJECT_ID}.{DATASET}.{table_name}`"
    return client.query(query).to_dataframe()
print("Loading initial tables from BigQuery...")
df_companyaccounts = load_table("staging_companyaccounts")
df_leads = load_table("staging_leads")
df_contacts = load_table("staging_contacts")
df_projects = load_table("staging_projects")
df_meeting_notes = load_table("staging_meeting_notes")
print("Initial tables loaded.")

# --- Cell 2: Load and Merge Excel Data into df_leads ---
print("\n--- Running Cell 2: Load and Merge Excel Data into df_leads ---")
excel_file_path = 'Leads.xlsx' # Make sure this file is uploaded

if 'df_leads' in locals() and isinstance(df_leads, pd.DataFrame):
    try:
        df_new_lead_info = pd.read_excel(excel_file_path)
        print(f"Successfully loaded data from '{excel_file_path}'. Shape: {df_new_lead_info.shape}")
        original_df_leads_cols = set(df_leads.columns)
        df_leads = pd.merge(df_leads, df_new_lead_info, on='Id', how='left', suffixes=('', '_from_excel'))
        newly_added_cols = list(set(df_leads.columns) - original_df_leads_cols)
        print(f"Merge complete. df_leads new shape: {df_leads.shape}")
        print(f"Newly added/updated columns from Excel: {newly_added_cols}")

        if 'IsConverted' in df_leads.columns:
            print("\n'IsConverted' column (from Excel) is now in df_leads.")
            print("Value counts for 'IsConverted' (from Excel) in df_leads:")
            print(df_leads['IsConverted'].value_counts(dropna=False))
        else:
            print("\nWARNING: 'IsConverted' (from Excel) column not found in df_leads after merge.")
        if 'ConvertedAccountId' in df_leads.columns:
            print("\n'ConvertedAccountId' (from Excel) is now in df_leads.")
        else:
            print("\nWARNING: 'ConvertedAccountId' (from Excel) column not found in df_leads after merge.")
    except FileNotFoundError:
        print(f"Error: The Excel file '{excel_file_path}' was not found.")
    except Exception as e:
        print(f"An error occurred while loading or merging Excel file: {e}")
else:
    print("Error: df_leads DataFrame not found or not a DataFrame for Cell 2. Please ensure initial load from BQ is done.")


# --- Cell 3: Convert String NaNs ---
print("\n--- Running Cell 3: Convert String NaNs ---")
dataframes_dict_cell3 = {
    "Company Accounts": df_companyaccounts if 'df_companyaccounts' in locals() and isinstance(df_companyaccounts, pd.DataFrame) else None,
    "Leads": df_leads if 'df_leads' in locals() and isinstance(df_leads, pd.DataFrame) else None,
    "Contacts": df_contacts if 'df_contacts' in locals() and isinstance(df_contacts, pd.DataFrame) else None,
    "Projects": df_projects if 'df_projects' in locals() and isinstance(df_projects, pd.DataFrame) else None,
    "Meeting Notes": df_meeting_notes if 'df_meeting_notes' in locals() and isinstance(df_meeting_notes, pd.DataFrame) else None
}
missing_value_formats = ['nan', 'NaN', 'Nan', 'NAN', 'None', 'NONE', 'null', 'NULL', '', ' ', '--', 'N/A', 'n/a', 'NA', 'na']
for df_name, df_current in dataframes_dict_cell3.items():
    if df_current is None: continue
    for col in df_current.columns:
        if df_current[col].dtype == 'object':
            for fmt in missing_value_formats:
                df_current.loc[df_current[col] == fmt, col] = np.nan

# --- Cell 4: Define Columns to Drop ---
print("\n--- Running Cell 4: Define Columns to Drop ---")
cols_to_drop_companyaccounts = ['About_the_Company__c', 'Areas_of_Interest__c', 'IPI_Remarks__c', 'IPI_WAR_Rating__c', 'Main_Products_Services__c', 'Other_Sources__c', 'ParentId', 'Other_Primary_Industry__c', 'Ready__c', 'Revenue__c', 'Tech_Seeker_Type__c', 'Year_of_Last_Engagement__c']
cols_to_drop_leads = ['Areas_of_Interest__c', 'Company_Classification__c', 'Company_Profile__c', 'Drop_Reason__c', 'Drop_Remarks__c', 'IPI_Remarks__c', 'Industry', 'Tech_Expert_Name__c', 'Tech_Need_Title__c', 'Tech_Offer_Title__c', 'Type_of_Enquiry__c', 'Type_of_Service__c']
cols_to_drop_contacts = ['Country__c', 'CventEvents__ContactStub__c', 'Cvent_Reference_ID__c', 'Cvent_Registration_Type__c', 'Source__c']
cols_to_drop_meeting_notes = ['Additional_Information__c', 'Background_Information__c', 'Meeting_Note_Migration_Ref_ID__c', 'Minutes_of_Meetings__c']

def drop_columns_and_summarize_fixed(df, df_name, cols_to_drop):
    if df is None: print(f"DataFrame '{df_name}' is not loaded. Skipping drop."); return None
    print(f"\n--- Dropping Columns from: {df_name} ---")
    print(f"Original shape: {df.shape}")
    print(f"Columns in '{df_name}' BEFORE drop attempt: {df.columns.tolist()}")
    existing_cols_to_drop = [col for col in cols_to_drop if col in df.columns]
    if not existing_cols_to_drop:
        print(f"No specified columns from the drop list found in '{df_name}'.")
        df_modified = df.copy()
    else:
        df_modified = df.drop(columns=existing_cols_to_drop, errors='ignore')
        print(f"Dropped columns: {existing_cols_to_drop}")
    print(f"New shape: {df_modified.shape}")
    return df_modified

if 'df_companyaccounts' in locals(): df_companyaccounts = drop_columns_and_summarize_fixed(df_companyaccounts, "Company Accounts", cols_to_drop_companyaccounts)
if 'df_leads' in locals(): df_leads = drop_columns_and_summarize_fixed(df_leads, "Leads", cols_to_drop_leads)
if 'df_contacts' in locals(): df_contacts = drop_columns_and_summarize_fixed(df_contacts, "Contacts", cols_to_drop_contacts)
if 'df_projects' in locals() and df_projects is not None:
    print(f"\n--- Processing DataFrame for Keep: Projects ---")
    cols_to_keep_projects = []
    possible_lead_fk_cols = ['LeadId', 'Lead__c', 'Created_From_Lead__c', 'Lead']
    possible_conversion_cols = ['IsWon', 'is_converted', 'iswon']
    if 'Id' in df_projects.columns: cols_to_keep_projects.append('Id')
    lead_fk_found = any(col_name in df_projects.columns for col_name in possible_lead_fk_cols)
    conversion_col_found = any(col_name in df_projects.columns for col_name in possible_conversion_cols)
    if lead_fk_found: cols_to_keep_projects.extend([col for col in possible_lead_fk_cols if col in df_projects.columns])
    if conversion_col_found: cols_to_keep_projects.extend([col for col in possible_conversion_cols if col in df_projects.columns])
    cols_to_keep_projects = sorted(list(set(cols_to_keep_projects)))
    if 'Id' in cols_to_keep_projects and \
       any(fk in cols_to_keep_projects for fk in possible_lead_fk_cols) and \
       any(conv in cols_to_keep_projects for conv in possible_conversion_cols):
        print(f"Identified columns to keep in Projects: {cols_to_keep_projects}")
        df_projects = df_projects[cols_to_keep_projects]
    else:
        print(f"WARNING: Could not robustly identify all essential columns in df_projects. Kept: {cols_to_keep_projects}. Review manually.")
    print(f"New shape of Projects: {df_projects.shape}")
else:
    print("df_projects not available for column keeping.")
if 'df_meeting_notes' in locals(): df_meeting_notes = drop_columns_and_summarize_fixed(df_meeting_notes, "Meeting Notes", cols_to_drop_meeting_notes)

# --- Cell 5: Updated Missing Value Analysis ---
print("\n--- Running Cell 5: Updated Missing Value Analysis ---")
dataframes_dict_cell5 = {
    "Company Accounts": df_companyaccounts if 'df_companyaccounts' in locals() else None,
    "Leads": df_leads if 'df_leads' in locals() else None,
    "Contacts": df_contacts if 'df_contacts' in locals() else None,
    "Projects": df_projects if 'df_projects' in locals() else None,
    "Meeting Notes": df_meeting_notes if 'df_meeting_notes' in locals() else None
}
for df_name, df_current in dataframes_dict_cell5.items():
    if df_current is None: continue
    print(f"\n--- Missing Value Analysis for {df_name} (Shape: {df_current.shape}) ---")
    missing_values = df_current.isnull().sum()
    missing_cols = missing_values[missing_values > 0]
    if not missing_cols.empty:
        print("Columns with missing values (count, %):")
        for col, count in missing_cols.items():
            print(f"  {col}: {count} ({ (count/len(df_current)*100):.2f}%)")
    else:
        print("No missing values found.")

# --- Cell 6: Row Deletion for False Company Names ---
print("\n--- Running Cell 6: Row Deletion for False Company Names ---")
false_company_names_to_delete = ["83", "99", "111234", "-", "[Company]"]
if 'df_leads' in locals() and df_leads is not None:
    leads_company_col = 'Company'
    if leads_company_col in df_leads.columns:
        original_lead_count = len(df_leads)
        df_leads[leads_company_col] = df_leads[leads_company_col].astype(str)
        df_leads = df_leads[~df_leads[leads_company_col].isin(false_company_names_to_delete)]
        print(f"df_leads: Removed {original_lead_count - len(df_leads)} rows with false company names. New shape: {df_leads.shape}")
    else: print(f"Column '{leads_company_col}' not found in df_leads.")
else: print("df_leads not available for Cell 6.")

if 'df_companyaccounts' in locals() and df_companyaccounts is not None:
    accounts_name_col = 'Name'
    if accounts_name_col in df_companyaccounts.columns:
        original_account_count = len(df_companyaccounts)
        df_companyaccounts[accounts_name_col] = df_companyaccounts[accounts_name_col].astype(str)
        df_companyaccounts = df_companyaccounts[~df_companyaccounts[accounts_name_col].isin(false_company_names_to_delete)]
        print(f"df_companyaccounts: Removed {original_account_count - len(df_companyaccounts)} rows with false names. New shape: {df_companyaccounts.shape}")
    else: print(f"Column '{accounts_name_col}' not found in df_companyaccounts.")
else: print("df_companyaccounts not available for Cell 6.")


# --- Cell 7: Merging Account Info into df_leads ---
print("\n--- Running Cell 7: Merging Account Info into df_leads ---")
if ('df_leads' in locals() and df_leads is not None) and \
   ('df_companyaccounts' in locals() and df_companyaccounts is not None):
    accounts_pk_col = 'Id'
    accounts_name_col = 'Name'
    leads_company_name_col = 'Company'
    possible_leads_fk_to_accounts = ['ConvertedAccountId', 'AccountId', 'Account_Id']
    actual_leads_fk_col = None
    for col_name in possible_leads_fk_to_accounts:
        if col_name in df_leads.columns: actual_leads_fk_col = col_name; break

    if actual_leads_fk_col: print(f"Found Account ID foreign key in df_leads: '{actual_leads_fk_col}'")
    else: print(f"WARNING: No prioritized Account ID FK found in df_leads from {possible_leads_fk_to_accounts}.")

    cols_from_accounts_to_merge = ['Id', 'Industry', 'Primary_Industry__c', 'Country__c', 'Company_Profile__c', 'Year_of_First_Engagement__c', 'AccountSource']
    cols_from_accounts_to_merge = [col for col in cols_from_accounts_to_merge if col in df_companyaccounts.columns]
    df_accounts_subset = df_companyaccounts[cols_from_accounts_to_merge].copy()

    if actual_leads_fk_col and accounts_pk_col in df_accounts_subset.columns:
        print(f"Attempting ID-based merge: df_leads['{actual_leads_fk_col}'] with df_companyaccounts['{accounts_pk_col}']")
        df_leads = pd.merge(df_leads, df_accounts_subset, left_on=actual_leads_fk_col, right_on=accounts_pk_col, how='left', suffixes=('', '_FromAccount'))
        print(f"ID-based merge complete. df_leads shape: {df_leads.shape}")
    else:
        print("Skipped ID-based merge. Conditions not met (e.g., FK missing in leads or PK in accounts).")

    if leads_company_name_col in df_leads.columns and accounts_name_col in df_companyaccounts.columns:
        print("Attempting name-based join/fill for remaining account info...")
        df_leads.loc[:, 'Clean_Lead_Company_Name'] = df_leads[leads_company_name_col].astype(str).str.lower().str.strip()
        df_companyaccounts.loc[:, 'Clean_Account_Name'] = df_companyaccounts[accounts_name_col].astype(str).str.lower().str.strip()
        df_accounts_for_map = df_companyaccounts.drop_duplicates(subset=['Clean_Account_Name'], keep='first')
        for acc_col in cols_from_accounts_to_merge:
            if acc_col == accounts_pk_col: continue
            target_col_in_leads = acc_col + '_FromAccount'
            if target_col_in_leads not in df_leads.columns: df_leads[target_col_in_leads] = np.nan
            df_leads[target_col_in_leads] = df_leads[target_col_in_leads].astype('object')
            if acc_col in df_accounts_for_map.columns:
                name_to_feature_map = df_accounts_for_map.set_index('Clean_Account_Name')[acc_col]
                condition_to_fill = df_leads[target_col_in_leads].isnull()
                if condition_to_fill.any():
                    values_to_map = df_leads.loc[condition_to_fill, 'Clean_Lead_Company_Name'].map(name_to_feature_map)
                    df_leads.loc[condition_to_fill, target_col_in_leads] = df_leads.loc[condition_to_fill, target_col_in_leads].fillna(values_to_map)
        if 'Clean_Lead_Company_Name' in df_leads.columns: df_leads.drop(columns=['Clean_Lead_Company_Name'], inplace=True)
    if accounts_pk_col + '_FromAccount' in df_leads.columns and actual_leads_fk_col != accounts_pk_col + '_FromAccount':
        df_leads = df_leads.drop(columns=[accounts_pk_col + '_FromAccount'])
    print("Account info merge attempt complete.")
else:
    print("Error: df_leads or df_companyaccounts not found for Cell 7.")

# --- Cells 8, 9, 10, 11: Imputations ---
print("\n--- Running Cell 8: Impute df_leads ---")
if 'df_leads' in locals() and df_leads is not None:
    cols_to_impute_leads = {'Address_country': 'Unknown_Country', 'Company': 'Unknown_Company', 'LeadSource': 'Unknown_Source'}
    for col, fill_val in cols_to_impute_leads.items():
        if col in df_leads.columns and df_leads[col].isnull().any():
            df_leads[col] = df_leads[col].fillna(fill_val)
else: print("df_leads not found for Cell 8.")

print("\n--- Running Cell 9: Impute df_companyaccounts ---")
if 'df_companyaccounts' in locals() and df_companyaccounts is not None:
    cols_to_impute_cat_accounts = {'Name': 'Unknown_Account_Name', 'Company_Profile__c': 'Unknown_Profile', 'Country__c': 'Unknown_Country', 'Nature_of_Business__c': 'Unknown_Nature', 'Industry': 'Unknown_Industry', 'Primary_Industry__c': 'Unknown_Primary_Industry', 'AccountSource': 'Unknown_Source', 'Lead_Source__c': 'Unknown_Source'}
    for col, fill_val in cols_to_impute_cat_accounts.items():
        if col in df_companyaccounts.columns and df_companyaccounts[col].isnull().any():
            df_companyaccounts[col] = df_companyaccounts[col].fillna(fill_val)
    if 'Year_of_First_Engagement__c' in df_companyaccounts.columns and df_companyaccounts['Year_of_First_Engagement__c'].isnull().any():
        df_companyaccounts['Year_of_First_Engagement__c'] = pd.to_numeric(df_companyaccounts['Year_of_First_Engagement__c'], errors='coerce')
        df_companyaccounts['Year_of_First_Engagement__c'] = df_companyaccounts['Year_of_First_Engagement__c'].fillna(df_companyaccounts['Year_of_First_Engagement__c'].median())
else: print("df_companyaccounts not found for Cell 9.")

print("\n--- Running Cell 10: Impute df_contacts ---")
if 'df_contacts' in locals() and df_contacts is not None:
    cols_to_impute_contacts = {'Areas_of_Interest__c': 'Unknown_Interest', 'AccountId': 'Unknown_Account_FK', 'LeadSource': 'Unknown_Source'}
    for col, fill_val in cols_to_impute_contacts.items():
        if col in df_contacts.columns and df_contacts[col].isnull().any():
            df_contacts[col] = df_contacts[col].fillna(fill_val)
else: print("df_contacts not found for Cell 10.")

print("\n--- Running Cell 11: Impute df_meeting_notes ---")
if 'df_meeting_notes' in locals() and df_meeting_notes is not None:
    cols_to_impute_meeting = {'Company_Country__c': 'Unknown_Country', 'Company_Needs__c': 'Unknown_Needs', 'Follow_Up_Action__c': 'No_Action_Specified', 'Lead__c': 'Unknown_Lead_FK', 'Organisation__c': 'Unknown_Organisation_FK', 'Potential__c': 'Unknown_Potential', 'RecordTypeId': 'Unknown_RecordType', 'Source__c': 'Unknown_Source'}
    for col, fill_val in cols_to_impute_meeting.items():
        if col in df_meeting_notes.columns and df_meeting_notes[col].isnull().any():
            df_meeting_notes[col] = df_meeting_notes[col].fillna(fill_val)
    if 'Lead__c' in df_meeting_notes.columns and (df_meeting_notes['Lead__c'] == 'Unknown_Lead_FK').sum() / len(df_meeting_notes) > 0.9:
         print("WARNING: 'Lead__c' in Meeting Notes is still overwhelmingly 'Unknown_Lead_FK'.")
else: print("df_meeting_notes not found for Cell 11.")

# --- Final Cell (Feature Engineering, Target Variable, Encoding - REVISED TARGET LOGIC) ---
print("\n--- Running Final Cell: Feature Engineering, Target Creation, Encoding ---")

# Initialize variables that might be defined conditionally
projects_lead_fk_col = None
projects_target_col = None
target_col_in_master = None # For the suffixed version from project merge

if 'df_leads' in locals() and df_leads is not None:
    # A. Date Features
    if 'CreatedDate' in df_leads.columns:
        if not pd.api.types.is_datetime64_any_dtype(df_leads['CreatedDate']):
            df_leads['CreatedDate'] = pd.to_datetime(df_leads['CreatedDate'], errors='coerce')
        if 'Lead_Created_Year' not in df_leads.columns:
            df_leads['Lead_Created_Year'] = df_leads['CreatedDate'].dt.year
            df_leads['Lead_Created_Month'] = df_leads['CreatedDate'].dt.month
            df_leads['Lead_Created_DayOfWeek'] = df_leads['CreatedDate'].dt.dayofweek
            print("Added Lead_Created_Year, Month, DayOfWeek to df_leads.")

    # B. Account Tenure
    account_tenure_col_name = 'Year_of_First_Engagement__c_FromAccount'
    if account_tenure_col_name in df_leads.columns and 'Lead_Created_Year' in df_leads.columns:
        df_leads[account_tenure_col_name] = pd.to_numeric(df_leads[account_tenure_col_name], errors='coerce')
        df_leads['Account_Tenure_At_Lead_Creation'] = df_leads['Lead_Created_Year'] - df_leads[account_tenure_col_name]
        df_leads.loc[df_leads['Account_Tenure_At_Lead_Creation'] < 0, 'Account_Tenure_At_Lead_Creation'] = 0
        print("Added 'Account_Tenure_At_Lead_Creation' to df_leads.")
    else:
        print(f"Could not create 'Account_Tenure_At_Lead_Creation': '{account_tenure_col_name}' or 'Lead_Created_Year' missing.")

    df_master = df_leads.copy()
    print(f"\nCreated df_master from df_leads. Shape: {df_master.shape}")

    # --- Target Variable Creation (Prioritizing 'IsConverted' from Excel) ---
    if 'IsConverted' in df_master.columns:
        print("Creating 'is_converted' target variable directly from 'IsConverted' (from Excel merge).")
        df_master['is_converted'] = df_master['IsConverted'].fillna(False).astype(bool).astype(int)
    elif 'df_projects' in locals() and df_projects is not None:
        print("WARNING: 'IsConverted' (from Excel) not in df_master. Attempting to use df_projects.")
        possible_lead_fk_cols_proj = ['Created_From_Lead__c', 'LeadId', 'Lead__c']
        for col in possible_lead_fk_cols_proj:
            if col in df_projects.columns: projects_lead_fk_col = col; break
        projects_target_col = 'IsWon'

        if projects_lead_fk_col and projects_target_col in df_projects.columns and 'Id' in df_master.columns:
            print(f"Attempting to merge df_projects into df_master using: df_master['Id'] and df_projects['{projects_lead_fk_col}']")
            df_projects_subset = df_projects[[projects_lead_fk_col, projects_target_col]].copy()
            df_master = pd.merge(df_master, df_projects_subset, left_on='Id', right_on=projects_lead_fk_col, how='left', suffixes=('', '_Project'))
            target_col_in_master = projects_target_col + '_Project' if projects_target_col + '_Project' in df_master.columns else projects_target_col
            if target_col_in_master in df_master.columns:
                df_master['is_converted'] = df_master[target_col_in_master].fillna(False).astype(bool).astype(int)
                print(f"Created 'is_converted' from df_projects '{target_col_in_master}'.")
            else:
                print("CRITICAL WARNING: Could not create target 'is_converted' from df_projects. Defaulting to 0.")
                df_master['is_converted'] = 0
        else:
            print("CRITICAL WARNING: Key columns missing for df_projects merge. Defaulting 'is_converted' to 0.")
            df_master['is_converted'] = 0
    else:
        print("CRITICAL WARNING: No source for target variable ('IsConverted' or df_projects). 'is_converted' defaulted to 0.")
        df_master['is_converted'] = 0

    print("'is_converted' value counts:")
    print(df_master['is_converted'].value_counts(dropna=False))

    # Clean up intermediate project columns if they exist from merge
    project_cols_merged = []
    for col_name_iter in df_master.columns:
        is_project_col = '_Project' in col_name_iter
        if projects_lead_fk_col is not None:
            is_project_col = is_project_col or (col_name_iter == projects_lead_fk_col)
        if projects_target_col is not None:
            is_project_col = is_project_col or (col_name_iter == projects_target_col)
        if target_col_in_master is not None and target_col_in_master != projects_target_col: # Check the suffixed version too
             is_project_col = is_project_col or (col_name_iter == target_col_in_master)
        if is_project_col:
            project_cols_merged.append(col_name_iter)

    project_cols_to_drop = [col for col in project_cols_merged if col != 'is_converted' and col != 'Id' and col != 'IsConverted']
    if project_cols_to_drop:
        df_master = df_master.drop(columns=project_cols_to_drop, errors='ignore')
        print(f"Cleaned up project-related columns: {project_cols_to_drop}")

    # --- Encoding Categorical Features ---
    print("\nEncoding Categorical Features...")
    categorical_features_to_encode = [
        'Industry', 'Address_country', 'LeadSource', 'Status', 'IAP__c', 'Lead_RT_Name__c',
        'Industry_FromAccount', 'Primary_Industry__c_FromAccount', 'Country__c_FromAccount',
        'Company_Profile__c_FromAccount', 'AccountSource_FromAccount'
    ]
    existing_categorical_features = [col for col in categorical_features_to_encode if col in df_master.columns]

    # Data Leakage Prevention for 'Status' column
    if 'Status' in existing_categorical_features:
        print("\n--- Inspecting 'Status' column before encoding for data leakage ---")
        status_unique_values = df_master['Status'].unique()
        print(f"Unique values in df_master['Status']: {status_unique_values}")
        # Define status values that represent a completed/final state (leakage)
        # Adjust this list based on your actual status values!
        problematic_status_values = ['Converted', 'Closed Won', 'Closed Lost', 'Disqualified - Final', 'Dropped - Final']

        leakage_found = any(status_val in status_unique_values for status_val in problematic_status_values)

        if leakage_found:
            print(f"WARNING: 'Status' column contains values ({problematic_status_values}) that might cause data leakage if one-hot encoded directly.")
            print("Removing 'Status' from the list of features to be one-hot encoded.")
            existing_categorical_features.remove('Status')
        else:
            print("'Status' column does not seem to contain obvious post-conversion leakage terms based on the check list.")

    if existing_categorical_features:
        print(f"Attempting to one-hot encode: {existing_categorical_features}")
        if 'IAP__c' in existing_categorical_features and pd.api.types.is_bool_dtype(df_master['IAP__c']):
            df_master['IAP__c'] = df_master['IAP__c'].astype(str)

        original_shape = df_master.shape
        df_master = pd.get_dummies(df_master, columns=existing_categorical_features, dummy_na=False, drop_first=False)
        print(f"Shape of df_master before encoding: {original_shape}")
        print(f"Shape of df_master after one-hot encoding: {df_master.shape}")
    else:
        print("No specified categorical features found in df_master for encoding.")
else:
    print("df_leads not available. Cannot proceed with final feature engineering and encoding.")

print("\n\n--- All Processing Steps From Notebook Applied with Fixes ---")
print("Review df_master, especially the 'is_converted' column and its value counts.")
print("Ensure the value counts for 'is_converted' show both 0s and 1s.")
print("The original 'Industry' column in df_master (from leads) can now be imputed using Gemini API if desired, before model training.")
print("Next main steps: Further feature selection/scaling, data splitting, and model training.")


Loading initial tables from BigQuery...
Initial tables loaded.

--- Running Cell 2: Load and Merge Excel Data into df_leads ---
Successfully loaded data from 'Leads.xlsx'. Shape: (47627, 6)
Merge complete. df_leads new shape: (47627, 25)
Newly added/updated columns from Excel: ['ConvertedOpportunityId', 'IsConverted', 'ConvertedContactId', 'ConvertedDate', 'ConvertedAccountId']

'IsConverted' column (from Excel) is now in df_leads.
Value counts for 'IsConverted' (from Excel) in df_leads:
IsConverted
False    46177
True      1450
Name: count, dtype: int64

'ConvertedAccountId' (from Excel) is now in df_leads.

--- Running Cell 3: Convert String NaNs ---

--- Running Cell 4: Define Columns to Drop ---

--- Dropping Columns from: Company Accounts ---
Original shape: (5631, 28)
Columns in 'Company Accounts' BEFORE drop attempt: ['Id', 'About_the_Company__c', 'Areas_of_Interest__c', 'Name', 'Company_Profile__c', 'RecordTypeId', 'Country__c', 'CreatedDate', 'IAP__c', 'IPI_Remarks__c', 'IPI_W

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.impute import SimpleImputer

# --- Assume X_train, X_test, y_train, y_test are from your data splitting step ---

if 'X_train' in locals() and 'X_test' in locals() and \
   'y_train' in locals() and 'y_test' in locals() and \
   isinstance(X_train, pd.DataFrame) and isinstance(X_test, pd.DataFrame):

    print("--- Step 0: Final Feature Preparation (Revised for String to Float Error) ---")

    X_train_model_ready = X_train.copy()
    X_test_model_ready = X_test.copy()

    # --- Step 1: Feature Scaling (for Numerical Features) ---
    print("\n--- Feature Scaling (for Numerical Features) ---")
    numerical_features_to_scale = [
        'Lead_Created_Year', 'Lead_Created_Month', 'Lead_Created_DayOfWeek',
        'Account_Tenure_At_Lead_Creation'
    ]
    numerical_features_present = [col for col in numerical_features_to_scale if col in X_train_model_ready.columns]

    if numerical_features_present:
        print(f"Identified numerical features for scaling: {numerical_features_present}")
        # Ensure these columns are purely numeric BEFORE scaling
        for col in numerical_features_present:
            if X_train_model_ready[col].dtype == 'object':
                X_train_model_ready[col] = pd.to_numeric(X_train_model_ready[col], errors='coerce')
                X_test_model_ready[col] = pd.to_numeric(X_test_model_ready[col], errors='coerce')
                # Impute NaNs created by coerce
                if X_train_model_ready[col].isnull().any():
                    median_val = X_train_model_ready[col].median() # Median from train
                    X_train_model_ready[col].fillna(median_val, inplace=True)
                    X_test_model_ready[col].fillna(median_val, inplace=True)
            elif X_train_model_ready[col].isnull().any(): # If already numeric but has NaNs
                    median_val = X_train_model_ready[col].median()
                    X_train_model_ready[col].fillna(median_val, inplace=True)
                    X_test_model_ready[col].fillna(median_val, inplace=True)


        scaler = StandardScaler()
        X_train_model_ready[numerical_features_present] = scaler.fit_transform(X_train_model_ready[numerical_features_present])
        X_test_model_ready[numerical_features_present] = scaler.transform(X_test_model_ready[numerical_features_present])
        print("Numerical features scaled successfully.")
    else:
        print("No explicitly listed numerical features found for scaling.")

    # --- Rigorous Check and Conversion/Dropping of Non-Numeric Columns BEFORE FIT ---
    print("\nPerforming rigorous check and handling of non-numeric columns in X_train_model_ready...")
    cols_to_drop_non_numeric = []
    for col in X_train_model_ready.columns:
        # If column is object, it means it wasn't one-hot encoded or a properly processed numeric feature
        if X_train_model_ready[col].dtype == 'object':
            print(f"WARNING: Column '{col}' is object type. It was not one-hot encoded or converted to numeric.")
            # Attempt to convert to numeric. If it fails (still object or many NaNs), mark for dropping.
            X_train_model_ready[col] = pd.to_numeric(X_train_model_ready[col], errors='coerce')
            if X_test_model_ready.get(col) is not None: # Ensure col exists in X_test_model_ready
                 X_test_model_ready[col] = pd.to_numeric(X_test_model_ready.get(col), errors='coerce')

            # If after coerce it's still object or mostly NaNs, it's problematic
            if X_train_model_ready[col].dtype == 'object' or X_train_model_ready[col].isnull().sum() > 0.9 * len(X_train_model_ready):
                cols_to_drop_non_numeric.append(col)
                print(f"  Marking '{col}' for dropping as it could not be reliably converted to numeric.")

        # Also check for datetime types explicitly
        elif pd.api.types.is_datetime64_any_dtype(X_train_model_ready[col]):
            cols_to_drop_non_numeric.append(col)
            print(f"  Marking datetime column '{col}' for dropping.")

    if cols_to_drop_non_numeric:
        print(f"Dropping problematic non-numeric/datetime columns: {list(set(cols_to_drop_non_numeric))}") # Use set to avoid duplicates
        X_train_model_ready = X_train_model_ready.drop(columns=list(set(cols_to_drop_non_numeric)))
        X_test_model_ready = X_test_model_ready.drop(columns=[col for col in list(set(cols_to_drop_non_numeric)) if col in X_test_model_ready.columns])
        print(f"Shape of X_train_model_ready after dropping non-numeric: {X_train_model_ready.shape}")
    else:
        print("No problematic non-numeric columns found needing drop before final NaN imputation.")

    # --- Final NaN Imputation Step for ALL remaining columns (should be numeric now) ---
    print("\nPerforming final NaN check and imputation on X_train_model_ready...")
    if X_train_model_ready.isnull().any().any():
        nan_columns_final = X_train_model_ready.columns[X_train_model_ready.isnull().any()].tolist()
        print(f"NaNs found in these columns before final imputation: {nan_columns_final}")

        # Ensure all columns are numeric before SimpleImputer
        for col in nan_columns_final:
            if not pd.api.types.is_numeric_dtype(X_train_model_ready[col]):
                 X_train_model_ready[col] = pd.to_numeric(X_train_model_ready[col], errors='coerce')
                 if col in X_test_model_ready.columns:
                    X_test_model_ready[col] = pd.to_numeric(X_test_model_ready[col], errors='coerce')

        imputer_final = SimpleImputer(strategy='median')
        X_train_model_ready = pd.DataFrame(imputer_final.fit_transform(X_train_model_ready), columns=X_train_model_ready.columns, index=X_train_model_ready.index)
        X_test_model_ready = pd.DataFrame(imputer_final.transform(X_test_model_ready), columns=X_test_model_ready.columns, index=X_test_model_ready.index)
        print(f"NaNs imputed using median for columns: {nan_columns_final}")
    else:
        print("No NaNs found in X_train_model_ready before fitting model.")

    print(f"Final NaN check in X_train_model_ready: {X_train_model_ready.isnull().sum().sum()}")


    print("\n--- Step 2: Training Logistic Regression Model ---")
    log_reg_model = LogisticRegression(
        class_weight='balanced',
        random_state=42,
        solver='liblinear',
        max_iter=1000
    )

    if X_train_model_ready.isnull().sum().sum() > 0:
        print("CRITICAL ERROR: NaNs still present in X_train_model_ready before fitting. Aborting training.")
        print(X_train_model_ready.isnull().sum()[X_train_model_ready.isnull().sum() > 0])
    else:
        print("Training the Logistic Regression model...")
        log_reg_model.fit(X_train_model_ready, y_train)
        print("Model training complete.")

        print("\n--- Step 3: Evaluating the Model ---")
        # Ensure X_test_model_ready has the same columns as X_train_model_ready
        X_test_model_ready = X_test_model_ready[X_train_model_ready.columns]

        y_pred_log_reg = log_reg_model.predict(X_test_model_ready)
        y_pred_proba_log_reg = log_reg_model.predict_proba(X_test_model_ready)[:, 1]

        # ... (Rest of your evaluation metric calculations and printouts) ...
        accuracy = accuracy_score(y_test, y_pred_log_reg)
        precision = precision_score(y_test, y_pred_log_reg)
        recall = recall_score(y_test, y_pred_log_reg)
        f1 = f1_score(y_test, y_pred_log_reg)
        roc_auc = roc_auc_score(y_test, y_pred_proba_log_reg)

        print("\nLogistic Regression Performance Metrics:")
        print(f"  Accuracy:  {accuracy:.4f}")
        print(f"  Precision: {precision:.4f}")
        print(f"  Recall:    {recall:.4f}")
        print(f"  F1-Score:  {f1:.4f}")
        print(f"  AUC-ROC:   {roc_auc:.4f}")

        print("\nConfusion Matrix:")
        cm = confusion_matrix(y_test, y_pred_log_reg)
        cm_df = pd.DataFrame(cm, index=['Actual Negative (0)', 'Actual Positive (1)'],
                             columns=['Predicted Negative (0)', 'Predicted Positive (1)'])
        print(cm_df)

        print("\nClassification Report:")
        print(classification_report(y_test, y_pred_log_reg))

        print("\n--- Feature Importances (Coefficients for Logistic Regression) ---")
        try:
            coefficients = pd.DataFrame({
                'Feature': X_train_model_ready.columns, # Use columns from the actual fitted data
                'Coefficient': log_reg_model.coef_[0]
            })
            coefficients['Absolute_Coefficient'] = coefficients['Coefficient'].abs()
            coefficients_sorted = coefficients.sort_values(by='Absolute_Coefficient', ascending=False)
            print("Top 20 most influential features:")
            print(coefficients_sorted.head(20))
        except Exception as e:
            print(f"Could not display coefficients: {e}")
else:
    print("Error: X_train, X_test, y_train, or y_test not found or not pandas DataFrames.")

--- Step 0: Final Feature Preparation (Revised for String to Float Error) ---

--- Feature Scaling (for Numerical Features) ---
Identified numerical features for scaling: ['Lead_Created_Year', 'Lead_Created_Month', 'Lead_Created_DayOfWeek', 'Account_Tenure_At_Lead_Creation']
Numerical features scaled successfully.

Performing rigorous check and handling of non-numeric columns in X_train_model_ready...
  Marking 'Status' for dropping as it could not be reliably converted to numeric.
Dropping problematic non-numeric/datetime columns: ['Status']
Shape of X_train_model_ready after dropping non-numeric: (26086, 295)

Performing final NaN check and imputation on X_train_model_ready...
NaNs found in these columns before final imputation: ['Year_of_First_Engagement__c_FromAccount']


<ipython-input-6-f8d1b8813bc5>:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train_model_ready[col].fillna(median_val, inplace=True)
<ipython-input-6-f8d1b8813bc5>:43: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)',

NaNs imputed using median for columns: ['Year_of_First_Engagement__c_FromAccount']
Final NaN check in X_train_model_ready: 0

--- Step 2: Training Logistic Regression Model ---
Training the Logistic Regression model...
Model training complete.

--- Step 3: Evaluating the Model ---

Logistic Regression Performance Metrics:
  Accuracy:  0.9951
  Precision: 0.9108
  Recall:    0.9862
  F1-Score:  0.9470
  AUC-ROC:   0.9992

Confusion Matrix:
                     Predicted Negative (0)  Predicted Positive (1)
Actual Negative (0)                    6204                      28
Actual Positive (1)                       4                     286

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6232
           1       0.91      0.99      0.95       290

    accuracy                           1.00      6522
   macro avg       0.96      0.99      0.97      6522
weighted avg       1.00      1.00      1.00      6522


-